Koristeći PyCuda okurženje napisati CUDA program za (matrično) množenje matrica.

1. Program koji vrši množenja matrica malih dimenzija (množenje se može izvršiti upotrebom jednom bloka niti).
2. Program koji vrši množenje matrica većih dimenzija, upotrebom većeg broja CUDA blokova.
3. Ubrzati rešenje iz stavke 2 upotrebom deljene memorije (tako da niti jednog bloka prvo dovuku deo podataka u deljenu memeorju, a potom sve čitaju iz deljene memorije).

In [ ]:
!pip install pycuda

In [2]:
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
import numpy as np
import math

1. Program koji vrši množenja matrica malih dimenzija (množenje se može izvršiti upotrebom jednom bloka niti).

In [13]:
mod = SourceModule(
    """
    __constant__ unsigned int uiRowLength, uiColumnLength;

    __global__ void vSingleBlockMatMul( int* piA, int* piB, int* piProduct )
    {
        float fSum = 0.0;

        for( int i = 0; i < uiRowLength; i++ )
        {
            fSum += piA[ threadIdx.x * uiRowLength + i ] * piB[ threadIdx.y + uiColumnLength * i ];
        }
        piProduct[ threadIdx.x * uiColumnLength + threadIdx.y ] = fSum;
    }
         
    """
)

In [ ]:
a = np.random.randn( 100, 250 ).astype( dtype = np.int32 )
b = np.random.randn( 250, 10 ).astype( dtype = np.int32 )

result = np.matmul( a, b )

c = np.zeros_like( result )

a_gpu = cuda.mem_alloc( a.nbytes )
b_gpu = cuda.mem_alloc( b.nbytes )
c_gpu = cuda.mem_alloc( c.nbytes )

cuda.memcpy_htod( a_gpu, a )
cuda.memcpy_htod( b_gpu, b )
cuda.memcpy_htod( c_gpu, c )

uiRowLength_gpu = mod.get_global( 'uiRowLength' )       # Index 0 means address, 1 is data length.
cuda.memcpy_htod( uiRowLength_gpu[0], np.uintc( a.shape[1] ) )

uiColumnLength_gpu = mod.get_global( 'uiColumnLength' ) # Index 0 means address, 1 is data length.
cuda.memcpy_htod( uiColumnLength_gpu[0], np.uintc( b.shape[1] ) )

gpu_vSingleBlockMatMul = mod.get_function( 'vSingleBlockMatMul' )
gpu_vSingleBlockMatMul( a_gpu, b_gpu, c_gpu, block = ( a.shape[0], b.shape[1], 1 ), grid = ( 1, 1, 1 ) )

cuda.memcpy_dtoh( c, c_gpu )

print('Correct: ', ( c == result ).all() )

2. Program koji vrši množenje matrica većih dimenzija, upotrebom većeg broja CUDA blokova.

In [6]:
mod = SourceModule(
    """
    __constant__ unsigned int uiRowLength, uiColumnLength, uiOutputMatrixWidth, uiOutputMatrixHeight;

    __global__ void vMultiBlockMatMul( int* piA, int* piB, int* piProduct )
    {

        const unsigned int uiProductIndex = threadIdx.x + blockDim.x * blockIdx.x + threadIdx.y * uiOutputMatrixWidth + blockDim.y * blockIdx.y * uiOutputMatrixHeight;
        const unsigned int uiRowIndexA = uiProductIndex / uiOutputMatrixWidth;
        const unsigned int uiColumnIndexB = uiProductIndex % uiOutputMatrixWidth;

        float fSum = 0.0;

        if ( uiProductIndex <= uiOutputMatrixWidth * uiOutputMatrixHeight )
        {
            for( int i = 0; i < uiRowLength; i++ )
            {
                fSum += piA[ uiRowIndexA * uiRowLength + i ] * piB[ uiColumnIndexB + i * uiOutputMatrixWidth ];
            }
            
            piProduct[ uiProductIndex ] = fSum;
        }
    }

    """
)

In [ ]:
a = np.random.randn( 1000, 1000 ).astype( dtype = np.int32 )
b = np.random.randn( 1000, 1000 ).astype( dtype = np.int32 )

result = np.matmul( a, b )

c = np.zeros_like( result )

a_gpu = cuda.mem_alloc( a.nbytes )
b_gpu = cuda.mem_alloc( b.nbytes )
c_gpu = cuda.mem_alloc( c.nbytes )

cuda.memcpy_htod( a_gpu, a )
cuda.memcpy_htod( b_gpu, b )
cuda.memcpy_htod( c_gpu, c )

uiRowLength_gpu = mod.get_global( 'uiRowLength' )                   # Index 0 means address, 1 is data length.
cuda.memcpy_htod( uiRowLength_gpu[0], np.uintc( a.shape[1] ) )

uiColumnLength_gpu = mod.get_global( 'uiColumnLength' )             # Index 0 means address, 1 is data length.
cuda.memcpy_htod( uiColumnLength_gpu[0], np.uintc( b.shape[1] ) )

uiOutputMatrixWidth = mod.get_global( 'uiOutputMatrixWidth' )       # Index 0 means address, 1 is data length.
cuda.memcpy_htod( uiOutputMatrixWidth[0], np.uintc( b.shape[1] ) )

uiOutputMatrixHeight = mod.get_global( 'uiOutputMatrixHeight' )     # Index 0 means address, 1 is data length.
cuda.memcpy_htod( uiOutputMatrixHeight[0], np.uintc( a.shape[0] ) )


gpu_vMultiBlockMatMul = mod.get_function( 'vMultiBlockMatMul' )
gpu_vMultiBlockMatMul( a_gpu, b_gpu, c_gpu, block = ( 32, 32, 1 ), grid = ( math.ceil( a.shape[1] / 32 ), math.ceil( a.shape[0] / 32 ), 1 ) )

cuda.memcpy_dtoh( c, c_gpu )

print('Correct: ', ( c == result ).all() )

3. Ubrzati rešenje iz stavke 2 upotrebom deljene memorije (tako da niti jednog bloka prvo dovuku deo podataka u deljenu memeorju, a potom sve čitaju iz deljene memorije).

In [10]:
mod = SourceModule(
    """
    __constant__ unsigned int uiRowLength, uiColumnLength, uiOutputMatrixWidth, uiOutputMatrixHeight;

    __global__ void vMultiBlockMatMulShared( int* piA, int* piB, int* piProduct )
    {
        const unsigned int uiGridHeight = gridDim.y * blockDim.y;
        const unsigned int uiGridWidth = gridDim.x * blockDim.x;

        const unsigned int uiCudaIndex = threadIdx.x + blockDim.x * blockIdx.x + threadIdx.y * uiGridWidth + blockDim.y * blockIdx.y * uiGridHeight;

        const unsigned int uiRowIndexA = uiCudaIndex / ( uiGridWidth );
        const unsigned int uiColumnIndexB = uiCudaIndex % ( uiGridWidth );

        if ( uiRowIndexA >= uiOutputMatrixHeight || uiColumnIndexB >= uiOutputMatrixWidth ) 
        {
            return;
        }

        __shared__ int piSubA[ 100 * 32 ];
        __shared__ int piSubB[ 100 * 32 ];

        if( threadIdx.x == 0 )
        {
            for( int i = 0; i < uiRowLength; i++ )
            {   
                piSubA[ threadIdx.y * uiRowLength + i ] = piA[ uiRowIndexA * uiRowLength + i ];
            }   
        }

        if( threadIdx.y == 0 )
        {
            for( int i = 0; i < uiRowLength; i++ )
            {
                piSubB[ i * blockDim.x + threadIdx.x ] = piB[ i * uiOutputMatrixWidth + uiColumnIndexB ];
            }   
        }        

        __syncthreads();
        
        int iSum = 0;

        for( int i = 0; i < uiRowLength; i++ )
        {
            iSum += piSubA[ threadIdx.y * uiRowLength + i ] * piSubB[ threadIdx.x + blockDim.x * i ];           
        }
        
        piProduct[ uiRowIndexA * uiOutputMatrixWidth + uiColumnIndexB ] = iSum;

    }
   """
   
)

In [ ]:
a = np.random.randn( 86, 10 ).astype( dtype = np.int32 )
b = np.random.randn( 10, 14 ).astype( dtype = np.int32 )
result = np.matmul( a, b )

c = np.ones_like( result )

a_gpu = cuda.mem_alloc( a.nbytes )
b_gpu = cuda.mem_alloc( b.nbytes )
c_gpu = cuda.mem_alloc( c.nbytes )

cuda.memcpy_htod( a_gpu, a )
cuda.memcpy_htod( b_gpu, b )
cuda.memcpy_htod( c_gpu, c )

uiRowLength_gpu = mod.get_global( 'uiRowLength' )                   # Index 0 means address, 1 is data length.
cuda.memcpy_htod( uiRowLength_gpu[0], np.uintc( a.shape[1] ) )

uiColumnLength_gpu = mod.get_global( 'uiColumnLength' )             # Index 0 means address, 1 is data length.
cuda.memcpy_htod( uiColumnLength_gpu[0], np.uintc( b.shape[1] ) )

uiOutputMatrixWidth = mod.get_global( 'uiOutputMatrixWidth' )       # Index 0 means address, 1 is data length.
cuda.memcpy_htod( uiOutputMatrixWidth[0], np.uintc( b.shape[1] ) )

uiOutputMatrixHeight = mod.get_global( 'uiOutputMatrixHeight' )     # Index 0 means address, 1 is data length.
cuda.memcpy_htod( uiOutputMatrixHeight[0], np.uintc( a.shape[0] ) )

block_size = 32

gpu_vMultiBlockMatMulShared = mod.get_function( 'vMultiBlockMatMulShared' )

grid_dim = max( math.ceil( a.shape[0] / block_size ), math.ceil( b.shape[1] / block_size ) )
gpu_vMultiBlockMatMulShared( a_gpu, b_gpu, c_gpu, block = ( block_size, block_size, 1 ), grid = ( grid_dim, grid_dim, 1 ) )

cuda.memcpy_dtoh( c, c_gpu )

print('Correct: ', ( c == result ).all() )